# Module 1 - Create a Blockchain


### To be installed:
### Flask==0.12.2: pip install Flask==0.12.2
### Postman HTTP Client: https://www.getpostman.com/

## Importing the libraries

In [1]:
import datetime
import hashlib
import json
from flask import Flask, jsonify

## Part 1 - Building a Blockchain

In [2]:
class Blockchain:

    def __init__(self):
        self.chain = []
        self.create_block(proof = 1, previous_hash = '0')

    def create_block(self, proof, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash}
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256\
            (str(new_proof**2 - previous_proof**2).\
             encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    
    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = \
                                   True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    
    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != \
                self.hash(previous_block):
                return False
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256\
            (str(proof**2 - previous_proof**2).encode()).\
            hexdigest()
            if hash_operation[:4] != '0000':
                return False
            previous_block = block
            block_index += 1
        return True


## Part 2 - Mining our Blockchain

### Creating a Web App

In [3]:
app = Flask(__name__)

## Creating a Blockchain

In [4]:
blockchain = Blockchain()

## Mining a new block

In [5]:
@app.route('/mine_block', methods = ['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    response = {'message': 'Congratulations, you just mined a block!',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash']}
    return jsonify(response), 200

## Getting the full Blockchain

In [6]:
@app.route('/get_chain', methods = ['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

## Checking if the Blockchain is valid

In [ ]:
@app.route('/is_valid', methods = ['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'All good. The Blockchain is valid.'}
    else:
        response = {'message': 'Houston, we have a problem. The Blockchain is not valid.'}
    return jsonify(response), 200

## Running the app

In [ ]:
app.run(host = '0.0.0.0', port = 5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Feb/2019 12:07:59] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:08:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Feb/2019 12:08:20] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:08:26] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:08:30] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:08:39] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:09:02] "GET /is_valid HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2019 12:09:34] "GET /get_chain HTTP/1.1" 200 -
